In [103]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
nlp = spacy.load('en')

In [4]:
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
caesar = gutenberg.raw('shakespeare-caesar.txt')

In [5]:
def text_cleaner(text):
    #Remove main header
    text = re.sub('[\[].*?[\]]', '', text)
    
    #Remove all the Act labels
    text = re.sub(r'Actus .[a-z]*\.', '', text)
    
    #Remove all the Scene labels
    text = re.sub(r'Scoena .[a-z]*\.', '', text)
    
    #Remove any double dashes
    text = re.sub(r'--', ' ', text)
    
    #Remove excess whitespace
    text = ' '.join(text.split())
    
    return text

In [6]:
hamlet_clean = text_cleaner(hamlet)
caesar_clean = text_cleaner(caesar)

In [7]:
hamlet_doc = nlp(hamlet_clean)
caesar_doc = nlp(caesar_clean)

In [8]:
hamlet_sent = [[sent, "Hamlet"] for sent in hamlet_doc.sents]
caesar_sent = [[sent, "Caesar"] for sent in caesar_doc.sents]

In [9]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(hamlet_sent + caesar_sent)
sentences.head()

,0,1
0,"(Enter, Barnardo, and, Francisco, two, Centine...",Hamlet
1,"(Barnardo, .)",Hamlet
2,"(Who, 's, there, ?)",Hamlet
3,"(Fran, .)",Hamlet
4,"(Nay, answer, me, :, Stand, &, vnfold, your, s...",Hamlet


In [10]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [11]:
#Set up bags
hamletwords = bag_of_words(hamlet_doc)
caesarwords = bag_of_words(caesar_doc)

#Combine bags to create a set of unique words.
common_words = set(hamletwords+caesarwords)

In [12]:
# Create our data frame with features.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,fountaine,breed,i'ue,spectacle,sodaine,trophees,y,vpon,water,cressant,...,feathers,practice,deale,leane,villaines,cobl,fearefull,clothe,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Enter, Barnardo, and, Francisco, two, Centine...",Hamlet
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Barnardo, .)",Hamlet
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Who, 's, there, ?)",Hamlet
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Fran, .)",Hamlet
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Nay, answer, me, :, Stand, &, vnfold, your, s...",Hamlet


In [13]:
word_counts.to_csv('SNM_words.csv')

In [84]:
#Stopword Counter
def stopword_count(sent):
    count = 0
    for word in sent:
        if str(word) in stopwords.words('english'):
            count+=1
    return count

#Add stopword count feature
def add_stopword_ct(sents, dataframe):
    counts = [stopword_count(token) for token in sents[0]]
    counter = pd.DataFrame(counts)
    counter.columns = ['stopwords']
    new_df = pd.concat((dataframe, counter), axis=1)
    return new_df

#Add sentence length feature
def add_sent_leng(sents, dataframe):
    lens = [len(token) for token in sents[0]]
    lengths = pd.DataFrame(lens)
    lengths.columns = ['length']
    new_df = pd.concat((dataframe, lengths), axis=1)
    return new_df

#Punctuation counter
def punct_count(token):
    count=0
    for word in token:
        if word.is_punct:
            count+=1
    return count

#Add punctuation count feature
def add_punc_count(sents, dataframe):
    counts = [punct_count(token) for token in sents[0]]
    punctuation = pd.DataFrame(counts)
    punctuation.columns = ['punctuation']
    new_df = pd.concat((dataframe, punctuation), axis=1)
    return new_df

#Unique word counter
def unique_wd(sent):
    words = [word for word in sent if not word.is_punct]
    u_words = set([token.text for token in words])
    return len(u_words)
        
#Add unique words feature
def add_unique_words(sents, dataframe):
    words = [unique_wd(token) for token in sents[0]]
    word_ct = pd.DataFrame(words)
    word_ct.columns = ['unique']
    new_df = pd.concat((dataframe, word_ct), axis=1)
    return new_df

In [42]:
sw_counts = add_stopword_ct(sentences, word_counts)

In [44]:
sw_len = add_sent_leng(sentences, sw_counts)

In [73]:
sw_punc = add_punc_count(sentences, sw_len)

In [85]:
sw_unique = add_unique_words(sentences, sw_punc)

In [89]:
#Testing Bag of Words
X = word_counts.drop(['text_sentence','text_source'],1)
Y = word_counts['text_source']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, random_state=1334)

In [95]:
rfc = ensemble.RandomForestClassifier(n_estimators=10, max_depth=5)
rfc.fit(X_train,y_train)
rfc_bow = cross_val_score(rfc, X_test, y_test, cv=10)
print(rfc_bow.mean())

0.6426313471474762


In [101]:
#Testing Handmade Features
X_hm = sw_unique.loc[:,['stopwords','length','punctuation','unique']]
Y_hm = word_counts['text_source']
Xhm_train, Xhm_test, yhm_train, yhm_test = train_test_split(X_hm,Y_hm,test_size=0.3, 
                                                            random_state=1334)


In [102]:
rfc_hm = ensemble.RandomForestClassifier(n_estimators=10, max_depth=5)
rfc_hm.fit(Xhm_train,yhm_train)
rfc_hm_bow = cross_val_score(rfc_hm, Xhm_test, yhm_test, cv=10)
print(rfc_hm_bow.mean())

0.5841119095957806


In [109]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [111]:
#Testing Tf-idf

hamletp=gutenberg.paras('shakespeare-hamlet.txt')
caesarp=gutenberg.paras('shakespeare-caesar.txt')

hamlet_para=[]
caesar_para=[]

for paragraph in hamletp:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    hamlet_para.append([' '.join(para), 'Hamlet'])
    
for paragraph in caesarp:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    caesar_para.append([' '.join(para), 'Caesar'])
                        
paragraphs = pd.DataFrame(hamlet_para+caesar_para)


In [118]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )
sents_tfidf = vectorizer.fit_transform(paragraphs.iloc[:,0])

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    sents_tfidf, paragraphs.iloc[:,1], test_size=0.3,random_state=1334)

In [119]:
rfc_tfidf = ensemble.RandomForestClassifier(n_estimators=10, max_depth=5)
rfc_tfidf.fit(X_train_tfidf, y_train_tfidf)
rfc_tfidf_score = cross_val_score(rfc_tfidf, X_test_tfidf,y_test_tfidf, cv=10)

In [120]:
rfc_tfidf_score.mean()

0.7736772247360482